In [1]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 8.3 MB 3.2 MB/s eta 0:00:01████████████████████▎| 8.1 MB 3.2 MB/s eta 0:00:01
     |████████████████████████████████| 114 kB 89.6 MB/s eta 0:00:01
     |████████████████████████████████| 96 kB 5.2 MB/s  eta 0:00:01
     |████████████████████████████████| 212 kB 95.7 MB/s eta 0:00:01
     |████████████████████████████████| 151 kB 92.5 MB/s eta 0:00:01
ERROR: google-cloud-core 1.3.0 has requirement google-api-core<2.0.0dev,>=1.16.0, but you'll have google-api-core 2.7.1 which is incompatible.
ERROR: google-api-core 2.7.1 has requirement google-auth<3.0dev,>=1.25.0, but you'll have google-auth 1.16.1 which is incompatible.


In [ ]:
from __future__ import print_function

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = '1SumpNwZL_e8h0xOlA5i3PSHVLVAt40B4-BYE30Ha-5E'
SAMPLE_RANGE_NAME = 'OE_D_EXP!I2:I6'


def main():
    """Shows basic usage of the Sheets API.
    Prints values from a sample spreadsheet.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'client_secret_918658031701-b2ncgpmcdso5m8jqr61ie1jjff8rl7in.apps.googleusercontent.com.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        service = build('sheets', 'v4', credentials=creds)

        # Call the Sheets API
        sheet = service.spreadsheets()
        result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                    range=SAMPLE_RANGE_NAME).execute()
        values = result.get('values', [])

        if not values:
            print('No data found.')
            return

        print('Name, Major:')
        for row in values:
            # Print columns A and E, which correspond to indices 0 and 4.
            print('%s, %s' % (row[0], row[4]))
    except HttpError as err:
        print(err)


if __name__ == '__main__':
    main()